In [ ]:
# -*- coding: utf-8 -*-
# install libraries
%pip install --upgrade tiktoken
%pip install --upgrade openai

In [4]:
# import libraries
import os
import time
import tiktoken
import concurrent.futures
import pprint
from openai import AzureOpenAI

# Funções que serão utilizadas durante os exemplos

In [15]:

# Nome dos modelos Azure OpenAI criados na subscrição 
GPT_35_TURBO = "gpt-35-turbo"
GPT_4 = "gpt-4"
GPT_4_32K = "gpt-4-32k"
GPT_4_TURBO = "gpt-4-turbo"
TEXT_EMBEDDING_ADA_002 = "text-embedding-ada-002"

# carregar tokenizador para os modelos de linguagem
encoding = tiktoken.get_encoding("cl100k_base")

# Exemplos de system messages
papel = {"bot": "Você é um assistente virtual, capaz de responder perguntas, fornecer informações e que sempre responde em formato JSON.",
        "escritor": "Você é um especialista em escrita e redação, capaz de escrever, compreender e analisar textos em português, especialmente textos opinativos e argumentativos.",
        "legislador": "Você é um especialista em legislação, capaz de escrever, interpretar e analisar leis e normas jurídicas em português.",
        "comediante": "Você é um comediante, capaz de criar piadas, contar histórias engraçadas e fazer humor em português.",
        "desevolvedor": "Você é um desenvolvedor de software, capaz de criar, testar e manter programas de computador em qualquer linguagem, mas em especial em Python, HTML e Javascript",
        "hacker": "Você é um hacker, capaz de invadir sistemas de computadores, roubar informações e burlar sistemas de segurança.",
        "professor": "Você é um professor, capaz de ensinar e explicar conceitos, teorias e práticas em qualquer disciplina. Com explicações claras e didáticas, com exemplos práticos e exercícios para fixação dos conceitos.",
        "instrutor": "Você é um instrutor, capaz de ensinar e treinar pessoas em qualquer área de conhecimento. Suas explicações são diretas, sucintas e sem rodeios.",
        "medico": "Você é um médico, capaz de diagnosticar, tratar e prevenir doenças em seres humanos e animais.",
        "advogado": "Você é um advogado, capaz de representar clientes em processos judiciais, elaborar contratos e prestar consultoria jurídica.",
        "padre": "Você é um padre, capaz de ministrar sacramentos, celebrar missas e aconselhar fiéis em questões espirituais.",
        "rabino": "Você é um rabino, capaz de ministrar cerimônias religiosas, ensinar a Torá e aconselhar fiéis em questões espirituais.",
        "pastor": "Você é um pastor, capaz de ministrar cultos, pregar a Bíblia e aconselhar fiéis em questões espirituais.",}


# inicialização do cliente Azure OpenAI
client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2024-02-15-preview"
)

# função para chamar o modelo de linguagem
def call_llm(deployment_name, system_message, question):
    start_time = time.perf_counter()
    response = client.chat.completions.create(
        model = deployment_name,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": question},
        ]
    )
    elapsed_time = time.perf_counter() - start_time # usado para calcular o tempo de execução
    resp = response.choices[0].message.content
    tokens_completion = num_tokens_from_string(resp, deployment_name) # conta os tokens da resposta
    # conta os tokens da mensagem do sistema e da pergunta (prompt)
    tokens_prompt = num_tokens_from_string(system_message, deployment_name) + num_tokens_from_string(question, deployment_name)
    # formatar a resposta da função
    return {"0.model": deployment_name, 
            "1.elapsed_time": elapsed_time, 
            "2.response": resp, 
            "3.num_tokens_completion": tokens_completion, 
            "4.num_tokens_prompt": tokens_prompt,
            "5.system_message": system_message,
            "6.question": question}

# funcao para contar tokens
def num_tokens_from_string(texto, model):
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = len(encoding.encode(texto))
    return num_tokens

# função para gerar embeddings
def generate_embeddings(text, model):
    start_time = time.perf_counter()
    embeddings = client.embeddings.create(input = [text], model=model).data[0].embedding
    elapsed_time = time.perf_counter() - start_time
    return embeddings, elapsed_time

# função para executar modelos simultaneamente
# models: lista de modelos
# system_message: mensagem do sistema
# question: pergunta
# retorna uma lista com os resultados
def execute_simultaneously(models, system_message, question, debug=False):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(call_llm, model, system_message, question) for model in models]
        results = []
        for future in concurrent.futures.as_completed(futures):
            if debug:
                print(future.result())
            results.append(future.result())
    return results

#funcao para imprimir resultados
def print_results(system_message, question, results):
    print("System message:", system_message)
    print("Question:", question)  
    for result in results:
        print("Model:", result["0.model"])
        print("Elapsed time:", result["1.elapsed_time"])
        print("Response:", result["2.response"])
        print("Num tokens completion:", result["3.num_tokens_completion"])
        print("Num tokens prompt:", result["4.num_tokens_prompt"])
        print("===========================================================\n")

In [14]:
def sample_agenda():
    agenda = '''12/03/2022 12:00:00 Ocupado
        12/03/2022 13:00:00 Ocupado
        12/03/2022 14:00:00 Ocupado
        12/03/2022 15:00:00 Livre
        12/03/2022 16:00:00 Ocupado
        12/03/2022 17:00:00 Livre
        12/03/2022 18:00:00 Ocupado
        12/03/2022 19:00:00 Livre
        12/03/2022 20:00:00 Ocupado'''
    return call_llm(GPT_4, papel["bot"], 
                    f"Com base nessa agenda, qual o próximo horário livre? {agenda}")

resposta = sample_agenda()
pprint.pprint(resposta)

{'0.model': 'gpt-4',
 '1.elapsed_time': 8.015733661000013,
 '2.response': '{\n  "ProximoHorarioLivre": "12/03/2022 15:00:00"\n}',
 '3.num_tokens_completion': 25,
 '4.num_tokens_prompt': 196,
 '5.system_message': 'Você é um assistente virtual, capaz de responder '
                     'perguntas, fornecer informações e que sempre responde em '
                     'formato JSON.',
 '6.question': 'Com base nessa agenda, qual o próximo horário livre? '
               '12/03/2022 12:00:00 Ocupado\n'
               '        12/03/2022 13:00:00 Ocupado\n'
               '        12/03/2022 14:00:00 Ocupado\n'
               '        12/03/2022 15:00:00 Livre\n'
               '        12/03/2022 16:00:00 Ocupado\n'
               '        12/03/2022 17:00:00 Livre\n'
               '        12/03/2022 18:00:00 Ocupado\n'
               '        12/03/2022 19:00:00 Livre\n'
               '        12/03/2022 20:00:00 Ocupado'}


# Exemplo de diferentes tons na resposta

In [11]:
def sample_system_message_intonation():
    system_message = {"despojado":"Você é jum assitente que sempre responde em tom despojado, de forma didática e em porguguês, apesar de eventualmente citar palavras em inglês.",
                      "formal":"Você é um assistente que sempre responde em tom formal, de forma didática e em português, sem citar palavras em inglês.",
                      "informal":"Você é um assistente que sempre responde em tom informal, de forma didática e em português, com gírias e expressões coloquiais."}
    question = "O que é inteligência artificial?"

    for item in system_message:
        print(item)
        print(call_llm(GPT_35_TURBO, system_message[item], "O que é inteligência artificial?"))

sample_system_message_intonation()

despojado
{'0.model': 'gpt-35-turbo', '1.elapsed_time': 1.896834402999957, '2.response': 'Inteligência artificial, ou IA, é a capacidade das máquinas de imitar funções cognitivas humanas, como aprendizado, raciocínio, resolução de problemas e até mesmo a compreensão da linguagem humana. Isso é feito por meio de algoritmos e modelos matemáticos que permitem que as máquinas tomem decisões, aprendam e se adaptem a novas situações. A ideia é criar sistemas que possam realizar tarefas que normalmente exigiriam inteligência humana, tais como reconhecimento de padrões, tomada de decisões, compreensão de linguagem natural, entre outras.', '3.num_tokens_completion': 150, '4.num_tokens_prompt': 49, '5.system_message': 'Você é jum assitente que sempre responde em tom despojado, de forma didática e em porguguês, apesar de eventualmente citar palavras em inglês.', '6.question': 'O que é inteligência artificial?'}
formal
{'0.model': 'gpt-35-turbo', '1.elapsed_time': 1.6978426469995611, '2.response':

# Exemplo de diferença entre modelos

In [16]:
system_message = papel["instrutor"]
question = "Conte-me um paradoxo de maneira estrudurada e sucinta com o mínimo de detalhes possível."
models = [GPT_35_TURBO, GPT_4, GPT_4_32K, GPT_4_TURBO]
results = execute_simultaneously(models, system_message, question, True)
print_results(system_message, question, results)

{'0.model': 'gpt-35-turbo', '1.elapsed_time': 1.475336480000351, '2.response': 'Paradoxo do mentiroso: "Esta afirmação é falsa." Se a afirmação é verdadeira, então ela é falsa, mas se ela for falsa, então ela é verdadeira. Isso gera uma contradição lógica.', '3.num_tokens_completion': 57, '4.num_tokens_prompt': 63, '5.system_message': 'Você é um instrutor, capaz de ensinar e treinar pessoas em qualquer área de conhecimento. Suas explicações são diretas, sucintas e sem rodeios.', '6.question': 'Conte-me um paradoxo de maneira estrudurada e sucinta com o mínimo de detalhes possível.'}
{'0.model': 'gpt-4-turbo', '1.elapsed_time': 12.520965451999928, '2.response': 'Paradoxo do Mentiroso:\n\n1. Premissa: Uma sentença afirma "Esta sentença é falsa".\n2. Análise: Se a sentença é verdadeira, então o que ela afirma é o caso – ela é falsa.\n3. Contradição: Mas se a sentença é falsa, então ela não é falsa conforme afirma, o que significa que a sentença seria verdadeira.\n4. Conclusão: A sentença 

# E aí, qual o seu prompt?

In [ ]:
# faça uma pergunta ao modelo, seja criativo!


prompt = ''' 
COLOQUE SEU PROMPT AQUI
'''

system_message = papel["professor"]

response = call_llm(GPT_4_32K, system_message, prompt)
response["2.response"]